In [1]:
%use fuel(2.3.1)

In [2]:
val envs = java.io.File("../../.env")
    .readLines()
    .map {
        it.split("=")[0] to it.split("=")[1].trim('"')
    }.toMap()

In [3]:
val session = envs.get("AOC_SESSION")
val year = 2021
val day = 4

In [4]:
fun getInput(year: Int, day: Int, session: String): String {
    val (_, _, result) = "https://adventofcode.com/$year/day/$day/input"
    .httpGet()
    .header("cookie" to "session=$session")
    .responseString()
        
    return result.get()
}

In [5]:
fun submitAnswer(year: Int, day: Int, session: String, level: Int, answer: String): String {
    val (_, _, result) = Fuel
    .post(
        "https://adventofcode.com/$year/day/$day/answer", 
        parameters = listOf("level" to level, "answer" to answer))
    .header("cookie" to "session=$session")
    .responseString()
        
    return result.get()
}

In [6]:
val sample = """7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7"""

In [7]:
val input = getInput(year, day, session)

In [8]:
fun solve(input: String, nth: Int): String {
    val lines = input.split('\n')
    val nums = lines[0].split(',').map(String::toInt)
    val boards = (2 until lines.size step 6).map {
        (it until it + 5).map {
            lines[it].split(' ').filter{it.length > 0}.map(String::toInt)
        }
    }
    val rank = if (nth >= 0) nth else boards.size + nth
    val mp = nums.withIndex().map {
        (index, num) -> num to index
    }.toMap()
    val winTurn = boards.map {
        board -> (0 until 5).map {
            i -> (0 until 5).map {
                mp.getOrDefault(board[i][it], 0)
            }.maxOf {it}
            .coerceAtMost((0 until 5).map {
                mp.getOrDefault(board[it][i], 0)
            }.maxOf {it})
        }.minOf {it} 
    }
    
    val boardOrderInWinTurn = (0 until boards.size).map {
        Pair(winTurn[it], it)
    }.toMutableList()
    boardOrderInWinTurn.sortBy{it.first}
    
    val winnerNo = boardOrderInWinTurn[rank].second
    val sumOfUnusedNumbers = boards[winnerNo].map {
        it.filter {
            mp.getOrDefault(it, 0) > winTurn[winnerNo]
        }.sum()
    }.sum()
        
    return (sumOfUnusedNumbers * nums[winTurn[winnerNo]]).toString()
}

In [9]:
fun partOne(input: String): String = solve(input, 0)

In [10]:
partOne(sample)

4512

In [11]:
val partOneAns = partOne(input)
partOneAns

49860

In [ ]:
submitAnswer(year, day, session, 1, partOneAns)

In [12]:
fun partTwo(input: String) = solve(input, -1)

In [13]:
partTwo(sample)

1924

In [14]:
val partTwoAns = partTwo(input)
partTwoAns

24628

In [ ]:
submitAnswer(year, day, session, 2, partTwoAns)